In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mytools as mt

SELECT TOP 50000 Title, 
                 Body, 
                 Tags, 
                 Id, 
                 Score, 
                 ViewCount, 
                 FavoriteCount, 
                 AnswerCount

FROM Posts

WHERE PostTypeId = 1 
      AND ViewCount > 10 
      AND AnswerCount > 0
      AND LEN(Tags) - LEN(REPLACE(Tags, '<','')) >= 5

ORDER BY Score DESC

In [8]:
data = pd.read_csv('data_stackoverflow.csv')
data.head()

,Title,Body,Tags,Id,Score,ViewCount,FavoriteCount,AnswerCount
0,Why is processing a sorted array faster than p...,"<p>In this C++ code, sorting the data (<em>bef...",<java><c++><performance><cpu-architecture><bra...,11227809,27235,1876578,0.0,25
1,How do I delete a Git branch locally and remot...,<p>Failed Attempts to Delete a Remote Branch:<...,<git><version-control><git-branch><git-push><g...,2003505,20387,11445451,0.0,41
2,What is the '-->' operator in C/C++?,"<p>After reading <a href=""http://groups.google...",<c++><c><operators><code-formatting><standards...,1642028,10151,1008628,0.0,26
3,"How do I force ""git pull"" to overwrite local f...",<p>How do I force an overwrite of local files ...,<git><version-control><overwrite><git-pull><gi...,1125968,9630,8458360,0.0,53
4,What and where are the stack and heap?,<ul>\n<li>What are the stack and heap?</li>\n<...,<data-structures><memory-management><heap-memo...,79923,9412,1927405,0.0,31


In [9]:
data.shape

(50000, 8)